In [1]:
from translation import FJSSPInstancesTranslator, SequenceGAEncoder
from model import ProductionEnvironment, Order
import pandas as pd

import re

In [ ]:
lit_path = r'C:\Users\huda\Documents\GitHub\scheduling_model\code\reworked_data_model\results\lit_results\lit_best_known.csv'
ga_path = r'C:\Users\huda\Downloads\newest\newest\\'
files = ['s0_s1_3600.txt','s4_3600.txt','s5_s6_s3_3600.txt', 's2a_3600.txt', 's2b_3600.txt', 's2c_s2d_3600.txt', 'behnke_geiger_repeat.txt', 'chambers.txt', 'dppaulli.txt', 'fattahi.txt', 'behnkegeiger.txt', 'brandimarte.txt']

gurobi_30 = r'C:\Users\huda\Documents\GitHub\scheduling_model\code\reworked_data_model\results\gurobi_results\formatted_results\30_min.txt'
gurobi_60 = r'C:\Users\huda\Documents\GitHub\scheduling_model\code\reworked_data_model\results\gurobi_results\formatted_results\60_min.txt'

benchmark_base_path = r'C:\Users\huda\Documents\GitHub\scheduling_model\code\external_test_data\FJSSPinstances\\'

In [ ]:
include_sdata = True

In [ ]:
def generate_one_order_per_recipe(production_environment : ProductionEnvironment) -> list[Order]:
    orders : list[Order] = []
    for i in range(len(production_environment.resources.values())): # should be the same amount as recipes for now
        orders.append(Order(delivery_time=1000, latest_acceptable_time=1000, resources=[(production_environment.get_resource(i), 1)], penalty=100.0, tardiness_fee=50.0, divisible=False, profit=500.0))
    return orders

In [ ]:
def read_benchmark_file(source, instance):
    if source.startswith('0'):
        target_file = f'Behnke{instance}.fjs'
    elif source.startswith('1'):
        target_file = f'BrandimarteMk{instance}.fjs'
    elif source.startswith('2a'):
        target_file = f'HurinkSdata{instance}.fjs'
    elif source.startswith('2b'):
        target_file = f'HurinkEdata{instance}.fjs'
    elif source.startswith('2c'):
        target_file = f'HurinkRdata{instance}.fjs'
    elif source.startswith('2d'):
        target_file = f'HurinkVdata{instance}.fjs'
    elif source.startswith('3'):
        target_file = f'DPpaulli{instance}.fjs'
    elif source.startswith('4'):
        target_file = f'ChambersBarnes{instance}.fjs'
    elif source.startswith('5'):
        target_file = f'Kacem{instance}.fjs'
    elif source.startswith('6'):
        target_file = f'Fattahi{instance}.fjs'
    path = benchmark_base_path + f'{source}\\{target_file}'
    return open(path, 'r').readlines()

In [ ]:
def load_benchmark_data(content):   
    line = dict()
    line['source'] = content[0]
    line['instance'] = int(content[1])

    # add information about the benchmark instance
    benchmark_file = read_benchmark_file(line['source'], line['instance'])
    benchmark_system = benchmark_file[0].split(' ')
    line['n_machines'] = int(benchmark_system[1])
    line['n_jobs'] = int(benchmark_system[0])
    line['average_workstations_per_operation'] = float(benchmark_system[2])
    jobs = benchmark_file[1:]
    line['n_operations'] = sum([int(x.split(' ')[0]) for x in jobs])
    line['average_operations_per_job'] = line['n_operations'] / line['n_jobs']
    line['beta_flexibility'] = line['average_workstations_per_operation']/line['n_machines']
    line['approximate_max_dissimilarity'] = line['n_operations'] + line['average_operations_per_job'] * line['average_workstations_per_operation']

    production_environment = FJSSPInstancesTranslator().translate(line['source'], line['instance'])
    orders = generate_one_order_per_recipe(production_environment)
    production_environment.orders = orders
    workstations_per_operation, base_durations, job_operations = SequenceGAEncoder().encode(production_environment, orders)
    unique_durations = []
    overall_amount_durations = []
    for duration in base_durations:
        for d in duration:
            if d not in unique_durations and d > 0:
                unique_durations.append(d)
        overall_amount_durations.extend([x for x in duration if x > 0])
    overall_amount_durations = len(overall_amount_durations)
    line['duration_variety'] = (len(unique_durations)/overall_amount_durations)
    return line    

In [ ]:
data = dict()
benchmark_data = []
for file in files:
    content = open(ga_path + file, 'r').readlines()
    for values in content:
        line = values.split(';')

        uid = line[0]+line[1]
        if include_sdata or not uid.startswith('2a'):
            if uid not in data:
                data[uid] = [float(line[6])]
            elif data[uid][0] > float(line[6]):
                data[uid] = [float(line[6])]

In [ ]:
content = open(lit_path, 'r').readlines()
for values in content[1:]:
    line = values.split(';')
    uid = line[0]+line[1]
    #if not any([x['source'] == line[0] and x['instance'] == line[1] for x in benchmark_data]):
    benchmark_data.append(load_benchmark_data(line))
    if include_sdata or not uid.startswith('2a'):
        if uid in data: # skip remaining benchmarks
            value = float(line[3])
            if value < 0:
                value = 10000000 # replacing missing data
            data[uid].append(value)

In [ ]:
content = open(gurobi_30, 'r').readlines()
for values in content[1:]:
    line = values.split(';')
    uid = line[0]+line[1]
    if include_sdata or not uid.startswith('2a'):
        if uid in data: # skip remaining benchmarks
            data[uid].append(int(float(line[2]) + 0.5))

In [ ]:
content = open(gurobi_60, 'r').readlines()
for values in content[1:]:
    line = values.split(';')
    uid = line[0]+line[1]
    if include_sdata or not uid.startswith('2a'):
        if uid in data: # skip remaining benchmarks
            data[uid][2] = (int(float(line[2]) + 0.5)) # assuming 60 min is better than the 30 min result

In [ ]:
def calc_gap(best, value):
    return (value - best)/best

In [5]:
example = '2a_Hurink_sdata29'
numbers = re.findall(r'\d+', example)
source = re.findall(r'[^\d+]', example)
print(numbers)
print(source)

['2', '29']
['a', '_', 'H', 'u', 'r', 'i', 'n', 'k', '_', 's', 'd', 'a', 't', 'a']


In [ ]:
benchmarks = dict()
mapping = dict()
for key in data.keys():
    numbers = re.findall(r'\d+', key)
    rest = re.findall(r'[^\d+]', key)
    sId = int(numbers[0])
    source = ''
    if sId == 0:
        source = '0_BehnkeGeiger'
    elif sId == 1:
        source = '1_Brandimarte'
    elif sId == 2:
        if rest[0] == 'a':
            source = '2a_Hurink_sdata'
        elif rest[0] == 'b':
            source = '2b_Hurink_edata'
        elif rest[0] == 'c':
            source = '2c_Hurink_rdata'
        else:
            source = '2d_Hurink_vdata'
    elif sId == 3:
        source = '3_DPpaulli'
    elif sId == 4:
        source = '4_ChambersBarnes'
    elif sId == 5:
        source = '5_Kacem'
    else:
        source = '6_Fattahi'
    instance = int(numbers[1])
    benchmarks[key] = load_benchmark_data(read_benchmark_file(source, instance))
    mapping[key] = (source, instance)

In [ ]:
# NOTE: 0 - GA, 1 - Lit, 2 - Gurobi
best = dict()
for key in data.keys():
    best[key] = min(data[key])
lit = dict()
for key in data.keys():
    lit[key] = data[1]
gaps_lit = dict()
for key in best.keys():
    gaps_lit[key] = dict()
    gaps_lit[key]['GA'] = (calc_gap(lit[key], data[key][0])) # GA
    #gaps_lit[key]['Literature'] = (calc_gap(lit[key], data[key][1])) # Lit
    gaps_lit[key]['Gurobi'] = (calc_gap(lit[key], data[key][2])) # Gurobi
gaps = dict()
for key in best.keys():
    gaps[key] = dict()
    gaps[key]['GA'] = (calc_gap(best[key], data[key][0])) # GA
    gaps[key]['Literature'] = (calc_gap(best[key], data[key][1])) # Lit
    gaps[key]['Gurobi'] = (calc_gap(best[key], data[key][2])) # Gurobi

In [ ]:
data_for_df = []
for key in data.keys():
    data_for_df.append({
        'Source': mapping[0],
        'Instance': mapping[1],
        'GA': data[0],
        'Gurobi': data[2],
        'Literature': data[1],
        'GA Best Gap': gaps[key]['GA'],
        'Gurobi Best Gap': gaps[key]['Gurobi'],
        'Literature Best Gap': gaps[key]['Literature'],
        'GA Literature Gap': gaps_lit[key]['GA'],
        'Gurobi Literature Gap': gaps_lit[key]['Gurobi'],
        'GA/Gurobi': '+' if data[0] < data[2] else 'o' if data[0] == data[2] else '-',
        'GA/Literature': '+' if data[0] < data[1] else 'o' if data[0] == data[1] else '-',
        'n_machines': benchmarks[key]['n_machines'],
        'n_operations': benchmarks[key]['n_operations'],
        'beta_flexibility': benchmarks[key]['beta_flexibility'],
        'duration_variety': benchmarks[key]['duration_variety']
    })

In [ ]:
df = pd.DataFrame(data_for_df)

In [ ]:
df.to_csv('all_results.csv', sep=';')